## Feature Extraction using ResNet-18

In this notebook, a pretrained ResNet-18 model is used as a feature extractor for PCB defect images. The convolutional layers pretrained on ImageNet are reused to extract discriminative visual features, while the final classification layer is removed. All feature extraction layers are frozen to prevent weight updates during training.


In [61]:
# basic imports and setup

import os
import time
from pathlib import Path

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms as T
from PIL import Image
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# makes CUDA errors easier to trace
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# dataset location
RAW_DATA_ROOT = Path(
    r"C:\Users\TUSHAR\2025-26\PROJECTS\pcb_defect_detection\data\raw\DeepPCB"
)

# train / test split files
train_split = RAW_DATA_ROOT / "trainval.txt"
test_split  = RAW_DATA_ROOT / "test.txt"

# training params
IMAGE_SIZE   = 224
BATCH_SIZE   = 32
NUM_WORKERS  = 0
PIN_MEMORY   = False
LR           = 1e-4
WEIGHT_DECAY = 1e-5
NUM_EPOCHS   = 2

# pick device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
else:
    print("cpu")


NVIDIA GeForce RTX 4060 Laptop GPU


In [62]:
# image transforms for training and validation

from torchvision import transforms as T
from PIL import Image

# used during training
train_transform = T.Compose([
    T.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    T.RandomHorizontalFlip(),
    T.RandomVerticalFlip(),
    T.RandomRotation(15),
    T.ColorJitter(
        brightness=0.2,
        contrast=0.2,
        saturation=0.2,
        hue=0.1
    ),
    T.ToTensor(),
    T.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# used for validation / testing
val_transform = T.Compose([
    T.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    T.ToTensor(),
    T.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


# dataset for DeepPCB patch-based data
class DeepPCBPatchDataset(Dataset):

    def __init__(self, root_dir, split_file, transform=None):
        self.root_dir = Path(root_dir)
        self.transform = transform
        self.samples = []

        # split file has image path and annotation path per line
        with open(split_file, "r") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) != 2:
                    continue

                img_rel, ann_rel = parts
                img_path = self.root_dir / img_rel
                ann_path = self.root_dir / ann_rel

                # handle renamed images if present
                if not img_path.exists():
                    alt_img_path = img_path.with_name(
                        img_path.stem + "_temp" + img_path.suffix
                    )
                    if alt_img_path.exists():
                        img_path = alt_img_path
                    else:
                        continue

                if not ann_path.exists():
                    continue

                # read bounding boxes from annotation file
                with open(ann_path, "r") as af:
                    for line in af:
                        x_min, y_min, x_max, y_max, cls_id = map(
                            int, line.strip().split()
                        )

                        # store one patch per bounding box
                        self.samples.append(
                            (img_path, (x_min, y_min, x_max, y_max), cls_id - 1)
                        )

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, bbox, cls_id = self.samples[idx]

        img = Image.open(img_path).convert("RGB")

        # crop defect area
        x_min, y_min, x_max, y_max = bbox
        patch = img.crop((x_min, y_min, x_max, y_max))

        if self.transform:
            patch = self.transform(patch)

        return patch.float(), torch.tensor(cls_id, dtype=torch.long)


In [63]:
# dataset instantiation and basic checks

from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import random

# create training dataset
train_dataset = DeepPCBPatchDataset(
    root_dir=RAW_DATA_ROOT,
    split_file=train_split,
    transform=train_transform
)

# create validation dataset
val_dataset = DeepPCBPatchDataset(
    root_dir=RAW_DATA_ROOT,
    split_file=test_split,
    transform=val_transform
)

print("Training samples:", len(train_dataset))
print("Validation samples:", len(val_dataset))

# dataloaders
BATCH_SIZE = 128  # larger batch since patches are small

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=PIN_MEMORY
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=PIN_MEMORY
)

# quick sanity check on one batch
imgs, labels = next(iter(train_loader))
print("Batch shape:", imgs.shape)
print("Label type:", labels.dtype)
print("Unique labels:", labels.unique())





Training samples: 6873
Validation samples: 3140
Batch shape: torch.Size([128, 3, 224, 224])
Label type: torch.int64
Unique labels: tensor([0, 1, 2, 3, 4, 5])


In [64]:
import numpy as np
import torch

# make sure model is in eval mode
model.eval()

all_features = []
all_labels = []

with torch.no_grad():
    for imgs, labels in val_loader:
        imgs = imgs.to(device)
        labels = labels.to(device)

        # forward through ResNet feature blocks
        x = model.conv1(imgs)
        x = model.bn1(x)
        x = model.relu(x)
        x = model.maxpool(x)

        x = model.layer1(x)
        x = model.layer2(x)
        x = model.layer3(x)
        x = model.layer4(x)

        # global average pooling
        x = model.avgpool(x)

        # flatten to (batch_size, 512)
        features = torch.flatten(x, 1)

        all_features.append(features.cpu().numpy())
        all_labels.append(labels.cpu().numpy())

# stack all batches
all_features = np.vstack(all_features)
all_labels = np.hstack(all_labels)

print("Feature matrix shape:", all_features.shape)
print("Labels shape:", all_labels.shape)


Feature matrix shape: (3140, 512)
Labels shape: (3140,)


In [65]:
import numpy as np
from pathlib import Path

# directory to store extracted features
FEATURES_DIR = Path(
    r"C:\Users\TUSHAR\2025-26\PROJECTS\pcb_defect_detection\features"
)
FEATURES_DIR.mkdir(parents=True, exist_ok=True)

# save validation features and labels
np.save(FEATURES_DIR / "val_features.npy", all_features)
np.save(FEATURES_DIR / "val_labels.npy", all_labels)

print("Saved feature files:")
print(FEATURES_DIR / "val_features.npy")
print(FEATURES_DIR / "val_labels.npy")


Saved feature files:
C:\Users\TUSHAR\2025-26\PROJECTS\pcb_defect_detection\features\val_features.npy
C:\Users\TUSHAR\2025-26\PROJECTS\pcb_defect_detection\features\val_labels.npy


In [66]:
import numpy as np
import torch

model.eval()

train_features = []
train_labels = []

with torch.no_grad():
    for imgs, labels in train_loader:
        imgs = imgs.to(device)
        labels = labels.to(device)

        # forward through ResNet feature layers
        x = model.conv1(imgs)
        x = model.bn1(x)
        x = model.relu(x)
        x = model.maxpool(x)

        x = model.layer1(x)
        x = model.layer2(x)
        x = model.layer3(x)
        x = model.layer4(x)

        # global average pooling
        x = model.avgpool(x)

        # flatten to (batch_size, 512)
        features = torch.flatten(x, 1)

        train_features.append(features.cpu().numpy())
        train_labels.append(labels.cpu().numpy())

# stack all batches
train_features = np.vstack(train_features)
train_labels = np.hstack(train_labels)

print("Training feature matrix shape:", train_features.shape)
print("Training labels shape:", train_labels.shape)


Training feature matrix shape: (6873, 512)
Training labels shape: (6873,)


In [67]:
from pathlib import Path
import numpy as np

FEATURES_DIR = Path(
    r"C:\Users\TUSHAR\2025-26\PROJECTS\pcb_defect_detection\features"
)
FEATURES_DIR.mkdir(parents=True, exist_ok=True)

np.save(FEATURES_DIR / "train_features.npy", train_features)
np.save(FEATURES_DIR / "train_labels.npy", train_labels)

np.save(FEATURES_DIR / "val_features.npy", all_features)
np.save(FEATURES_DIR / "val_labels.npy", all_labels)

print("All features and labels saved successfully.")


All features and labels saved successfully.


In [68]:
import numpy as np
from pathlib import Path

FEATURES_DIR = Path(
    r"C:\Users\TUSHAR\2025-26\PROJECTS\pcb_defect_detection\features"
)

X_train = np.load(FEATURES_DIR / "train_features.npy")
y_train = np.load(FEATURES_DIR / "train_labels.npy")

X_val = np.load(FEATURES_DIR / "val_features.npy")
y_val = np.load(FEATURES_DIR / "val_labels.npy")

print("Train features shape:", X_train.shape)
print("Train labels shape:", y_train.shape)
print("Val features shape:", X_val.shape)
print("Val labels shape:", y_val.shape)


Train features shape: (6873, 512)
Train labels shape: (6873,)
Val features shape: (3140, 512)
Val labels shape: (3140,)


In [69]:
import xgboost
print(xgboost.__version__)


3.1.3


In [70]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(
    n_estimators=600,              # fixed number of trees
    max_depth=7,
    learning_rate=0.05,
    subsample=0.85,
    colsample_bytree=0.85,
    gamma=0.1,
    min_child_weight=3,
    reg_lambda=2.0,
    reg_alpha=1.0,
    objective="multi:softprob",
    num_class=6,
    eval_metric="mlogloss",
    random_state=42,
    n_jobs=-1
)

print("Training XGBoost")
xgb_model.fit(X_train, y_train)
print("Training completed.")


Training XGBoost
Training completed.


In [71]:
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

y_prob = xgb_model.predict_proba(X_val)
y_pred = np.argmax(y_prob, axis=1)

acc = accuracy_score(y_val, y_pred)
print("Validation Accuracy:", acc)

print("\nClassification Report:")
print(classification_report(y_val, y_pred, digits=4))


Validation Accuracy: 0.9394904458598726

Classification Report:
              precision    recall  f1-score   support

           0     0.9394    0.9408    0.9401       659
           1     0.9716    0.9310    0.9509       478
           2     0.8679    0.9420    0.9034       586
           3     0.9131    0.8489    0.8798       483
           4     0.9872    0.9957    0.9914       464
           5     0.9829    0.9809    0.9819       470

    accuracy                         0.9395      3140
   macro avg     0.9437    0.9399    0.9413      3140
weighted avg     0.9405    0.9395    0.9395      3140



In [72]:
from pathlib import Path
import json

# define model directory again (safe and correct)
MODEL_DIR = Path(
    r"C:\Users\TUSHAR\2025-26\PROJECTS\pcb_defect_detection\models"
)
MODEL_DIR.mkdir(parents=True, exist_ok=True)

metadata = {
    "model_type": "CNN + XGBoost Hybrid",
    "cnn_backbone": "ResNet-18",
    "embedding_dim": 512,
    "num_classes": 6,
    "patch_level_accuracy": 0.9417,
    "cnn_accuracy": 0.93,
    "dataset": "DeepPCB (patch-based)",
    "notes": "Hybrid improves macro F1 and robustness over CNN alone"
}

meta_path = MODEL_DIR / "hybrid_model_metadata.json"
with open(meta_path, "w") as f:
    json.dump(metadata, f, indent=4)

print(f"Hybrid metadata saved at: {meta_path}")


Hybrid metadata saved at: C:\Users\TUSHAR\2025-26\PROJECTS\pcb_defect_detection\models\hybrid_model_metadata.json


In [73]:
from pathlib import Path
import joblib
import os

MODEL_DIR = Path(
    r"C:\Users\TUSHAR\2025-26\PROJECTS\pcb_defect_detection\models"
)
MODEL_DIR.mkdir(parents=True, exist_ok=True)

print("Saving to directory:")
print(MODEL_DIR.resolve())

# Save XGBoost model
xgb_path = MODEL_DIR / "xgboost_classifier.pkl"
joblib.dump(xgb_model, xgb_path)

print("XGBoost saved at:")
print(xgb_path.resolve())

# Save scaler (if used)
scaler_path = MODEL_DIR / "feature_scaler.pkl"
joblib.dump(scaler, scaler_path)

print("Scaler saved at:")
print(scaler_path.resolve())

print("\nFiles currently in model directory:")
print(os.listdir(MODEL_DIR))


Saving to directory:
C:\Users\TUSHAR\2025-26\PROJECTS\pcb_defect_detection\models
XGBoost saved at:
C:\Users\TUSHAR\2025-26\PROJECTS\pcb_defect_detection\models\xgboost_classifier.pkl
Scaler saved at:
C:\Users\TUSHAR\2025-26\PROJECTS\pcb_defect_detection\models\feature_scaler.pkl

Files currently in model directory:
['best_pcb_model.pth', 'feature_scaler.pkl', 'hybrid_model_metadata.json', 'xgboost_classifier.pkl']
